<a href="https://colab.research.google.com/github/TimTree/DS-Unit-2-Regression-Classification/blob/master/module3/assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `RidgeRegression`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `RidgeRegression`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv('../data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [6]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [7]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [0]:
# Convert SALE_DATE to datetime

df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])

In [9]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,2019-01-01
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,2019-01-01
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,2019-01-01
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,2019-01-01
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,2019-01-01


In [0]:
# Filter dataset to one family dwellings
df2 = df[df['BUILDING_CLASS_CATEGORY']=='01 ONE FAMILY DWELLINGS']

In [11]:
df2.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
7,2,OTHER,01 ONE FAMILY DWELLINGS,1,4090,37,NaN,A1,1193 SACKET AVENUE,NaN,10461.0,1.0,0.0,1.0,"3,404",1328.0,1925.0,1,A1,0,2019-01-01
8,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,18,NaN,A5,1215 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,2019-01-01
9,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,20,NaN,A5,1211 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,2019-01-01
42,3,OTHER,01 ONE FAMILY DWELLINGS,1,6809,54,NaN,A1,2601 AVENUE R,NaN,11229.0,1.0,0.0,1.0,"3,333",1262.0,1925.0,1,A1,0,2019-01-01
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,2019-01-01


In [12]:
df2.dtypes

BOROUGH                                   object
NEIGHBORHOOD                              object
BUILDING_CLASS_CATEGORY                   object
TAX_CLASS_AT_PRESENT                      object
BLOCK                                      int64
LOT                                        int64
EASE-MENT                                float64
BUILDING_CLASS_AT_PRESENT                 object
ADDRESS                                   object
APARTMENT_NUMBER                          object
ZIP_CODE                                 float64
RESIDENTIAL_UNITS                        float64
COMMERCIAL_UNITS                         float64
TOTAL_UNITS                              float64
LAND_SQUARE_FEET                          object
GROSS_SQUARE_FEET                        float64
YEAR_BUILT                               float64
TAX_CLASS_AT_TIME_OF_SALE                  int64
BUILDING_CLASS_AT_TIME_OF_SALE            object
SALE_PRICE                                 int64
SALE_DATE           

In [13]:
df2.shape

(5061, 21)

In [14]:
# Check for null values and drop them from the dataset
df2.isnull().sum()

BOROUGH                              0
NEIGHBORHOOD                         0
BUILDING_CLASS_CATEGORY              0
TAX_CLASS_AT_PRESENT                 0
BLOCK                                0
LOT                                  0
EASE-MENT                         5061
BUILDING_CLASS_AT_PRESENT            0
ADDRESS                              0
APARTMENT_NUMBER                  5059
ZIP_CODE                             0
RESIDENTIAL_UNITS                    0
COMMERCIAL_UNITS                     0
TOTAL_UNITS                          0
LAND_SQUARE_FEET                     0
GROSS_SQUARE_FEET                    0
YEAR_BUILT                           0
TAX_CLASS_AT_TIME_OF_SALE            0
BUILDING_CLASS_AT_TIME_OF_SALE       0
SALE_PRICE                           0
SALE_DATE                            0
dtype: int64

In [0]:
df2 = df2.drop(['EASE-MENT','APARTMENT_NUMBER'],axis=1)

In [16]:
df2.isnull().sum()

BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              0
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         0
ADDRESS                           0
ZIP_CODE                          0
RESIDENTIAL_UNITS                 0
COMMERCIAL_UNITS                  0
TOTAL_UNITS                       0
LAND_SQUARE_FEET                  0
GROSS_SQUARE_FEET                 0
YEAR_BUILT                        0
TAX_CLASS_AT_TIME_OF_SALE         0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
dtype: int64

In [0]:
# Split train and test data

train = df2[(df2['SALE_DATE'] >= '2019-01-01') & (df2['SALE_DATE'] <= '2019-03-31')]
test = df2[(df2['SALE_DATE'] >= '2019-04-01') & (df2['SALE_DATE'] <= '2019-04-30')]

In [18]:
train.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
7,2,OTHER,01 ONE FAMILY DWELLINGS,1,4090,37,A1,1193 SACKET AVENUE,10461.0,1.0,0.0,1.0,"3,404",1328.0,1925.0,1,A1,0,2019-01-01
8,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,18,A5,1215 VAN NEST AVENUE,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,2019-01-01
9,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,20,A5,1211 VAN NEST AVENUE,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,2019-01-01
42,3,OTHER,01 ONE FAMILY DWELLINGS,1,6809,54,A1,2601 AVENUE R,11229.0,1.0,0.0,1.0,"3,333",1262.0,1925.0,1,A1,0,2019-01-01
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,2019-01-01


In [19]:
test.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
18182,1,OTHER,01 ONE FAMILY DWELLINGS,1,1708,24,A9,271 PLEASANT AVENUE,10029.0,1.0,0.0,1.0,928,2501.0,1890.0,1,A9,1,2019-04-01
18235,2,OTHER,01 ONE FAMILY DWELLINGS,1,5913,878,A1,4616 INDEPENDENCE AVENUE,10471.0,1.0,0.0,1.0,"5,000",2272.0,1930.0,1,A1,895000,2019-04-01
18239,2,OTHER,01 ONE FAMILY DWELLINGS,1,5488,48,A2,558 ELLSWORTH AVENUE,10465.0,1.0,0.0,1.0,"2,500",720.0,1935.0,1,A2,253500,2019-04-01
18244,3,OTHER,01 ONE FAMILY DWELLINGS,1,5936,31,A1,16 BAY RIDGE PARKWAY,11209.0,1.0,0.0,1.0,"2,880",2210.0,1925.0,1,A1,1300000,2019-04-01
18269,3,OTHER,01 ONE FAMILY DWELLINGS,1,3896,33,A5,15 VAN SICLEN COURT,11207.0,1.0,0.0,1.0,"1,974",1400.0,1910.0,1,A5,0,2019-04-01


In [20]:
# Get categorical features
df2.describe(exclude=['number','datetime'])

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE
count,5061,5061,5061,5061,5061,5061,5061,5061
unique,5,9,1,3,16,5001,1469,12
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,277 NICHOLS AVENUE,"4,000",A1
freq,2561,4679,5061,5013,1930,4,498,1932


In [0]:
# One-hot encode

import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)

In [0]:
train = encoder.fit_transform(train)

In [23]:
train.head()

,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_ASTORIA,NEIGHBORHOOD_UPPER EAST SIDE (79-96),BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,TAX_CLASS_AT_PRESENT_1B,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A7,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,BUILDING_CLASS_AT_PRESENT_V0,ADDRESS_1193 SACKET AVENUE,ADDRESS_1215 VAN NEST AVENUE,ADDRESS_1211 VAN NEST AVENUE,ADDRESS_2601 AVENUE R,ADDRESS_4832 BAY PARKWAY,...,"LAND_SQUARE_FEET_4,298","LAND_SQUARE_FEET_1,060",LAND_SQUARE_FEET_946,"LAND_SQUARE_FEET_1,698","LAND_SQUARE_FEET_3,012","LAND_SQUARE_FEET_2,231","LAND_SQUARE_FEET_4,485","LAND_SQUARE_FEET_3,281","LAND_SQUARE_FEET_1,918","LAND_SQUARE_FEET_1,801","LAND_SQUARE_FEET_5,466","LAND_SQUARE_FEET_1,380","LAND_SQUARE_FEET_1,930","LAND_SQUARE_FEET_3,338","LAND_SQUARE_FEET_3,692","LAND_SQUARE_FEET_11,800","LAND_SQUARE_FEET_2,755","LAND_SQUARE_FEET_5,202","LAND_SQUARE_FEET_3,230","LAND_SQUARE_FEET_2,121","LAND_SQUARE_FEET_4,361","LAND_SQUARE_FEET_3,910","LAND_SQUARE_FEET_6,867",GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A7,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0,SALE_PRICE,SALE_DATE
7,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,4090,37,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1328.0,1925.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,2019-01-01
8,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,4120,18,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1728.0,1935.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2019-01-01
9,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,4120,20,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1728.0,1935.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2019-01-01
42,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,6809,54,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1262.0,1925.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,2019-01-01
44,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,5495,801,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1325.0,1930.0,1,0,0,1,0,0,0,0,0,0,0,0,0,550000,2019-01-01


In [0]:
test = encoder.transform(test)

In [25]:
test.head()

,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_ASTORIA,NEIGHBORHOOD_UPPER EAST SIDE (79-96),BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,TAX_CLASS_AT_PRESENT_1B,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A7,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,BUILDING_CLASS_AT_PRESENT_V0,ADDRESS_1193 SACKET AVENUE,ADDRESS_1215 VAN NEST AVENUE,ADDRESS_1211 VAN NEST AVENUE,ADDRESS_2601 AVENUE R,ADDRESS_4832 BAY PARKWAY,...,"LAND_SQUARE_FEET_4,298","LAND_SQUARE_FEET_1,060",LAND_SQUARE_FEET_946,"LAND_SQUARE_FEET_1,698","LAND_SQUARE_FEET_3,012","LAND_SQUARE_FEET_2,231","LAND_SQUARE_FEET_4,485","LAND_SQUARE_FEET_3,281","LAND_SQUARE_FEET_1,918","LAND_SQUARE_FEET_1,801","LAND_SQUARE_FEET_5,466","LAND_SQUARE_FEET_1,380","LAND_SQUARE_FEET_1,930","LAND_SQUARE_FEET_3,338","LAND_SQUARE_FEET_3,692","LAND_SQUARE_FEET_11,800","LAND_SQUARE_FEET_2,755","LAND_SQUARE_FEET_5,202","LAND_SQUARE_FEET_3,230","LAND_SQUARE_FEET_2,121","LAND_SQUARE_FEET_4,361","LAND_SQUARE_FEET_3,910","LAND_SQUARE_FEET_6,867",GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A7,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0,SALE_PRICE,SALE_DATE
18182,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,1708,24,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2501.0,1890.0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,2019-04-01
18235,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,5913,878,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2272.0,1930.0,1,1,0,0,0,0,0,0,0,0,0,0,0,895000,2019-04-01
18239,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,5488,48,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,720.0,1935.0,1,0,0,0,0,1,0,0,0,0,0,0,0,253500,2019-04-01
18244,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,5936,31,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2210.0,1925.0,1,1,0,0,0,0,0,0,0,0,0,0,0,1300000,2019-04-01
18269,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,3896,33,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1400.0,1910.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2019-04-01


In [26]:
train.isnull().sum()

BOROUGH_2                                          0
BOROUGH_3                                          0
BOROUGH_4                                          0
BOROUGH_5                                          0
BOROUGH_1                                          0
NEIGHBORHOOD_OTHER                                 0
NEIGHBORHOOD_FLUSHING-NORTH                        0
NEIGHBORHOOD_BOROUGH PARK                          0
NEIGHBORHOOD_EAST NEW YORK                         0
NEIGHBORHOOD_UPPER EAST SIDE (59-79)               0
NEIGHBORHOOD_BEDFORD STUYVESANT                    0
NEIGHBORHOOD_FOREST HILLS                          0
NEIGHBORHOOD_ASTORIA                               0
NEIGHBORHOOD_UPPER EAST SIDE (79-96)               0
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS    0
TAX_CLASS_AT_PRESENT_1                             0
TAX_CLASS_AT_PRESENT_1D                            0
TAX_CLASS_AT_PRESENT_1B                            0
BLOCK                                         

In [0]:
# Do feature selection with selectkbest

features = train.columns.drop(['SALE_DATE', 'SALE_PRICE'])
target = 'SALE_PRICE'

x_train = train[features]
y_train = train[target]

x_test = train[features]
y_test = test[target]

In [28]:
features

Index(['BOROUGH_2', 'BOROUGH_3', 'BOROUGH_4', 'BOROUGH_5', 'BOROUGH_1',
       'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'NEIGHBORHOOD_BOROUGH PARK', 'NEIGHBORHOOD_EAST NEW YORK',
       'NEIGHBORHOOD_UPPER EAST SIDE (59-79)',
       ...
       'BUILDING_CLASS_AT_TIME_OF_SALE_A9',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A3',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A2',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A0',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A7',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A4',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A6',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A8',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S0'],
      dtype='object', length=5393)

In [29]:
x_train

,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_ASTORIA,NEIGHBORHOOD_UPPER EAST SIDE (79-96),BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,TAX_CLASS_AT_PRESENT_1B,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A7,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,BUILDING_CLASS_AT_PRESENT_V0,ADDRESS_1193 SACKET AVENUE,ADDRESS_1215 VAN NEST AVENUE,ADDRESS_1211 VAN NEST AVENUE,ADDRESS_2601 AVENUE R,ADDRESS_4832 BAY PARKWAY,...,"LAND_SQUARE_FEET_2,260","LAND_SQUARE_FEET_5,005","LAND_SQUARE_FEET_4,298","LAND_SQUARE_FEET_1,060",LAND_SQUARE_FEET_946,"LAND_SQUARE_FEET_1,698","LAND_SQUARE_FEET_3,012","LAND_SQUARE_FEET_2,231","LAND_SQUARE_FEET_4,485","LAND_SQUARE_FEET_3,281","LAND_SQUARE_FEET_1,918","LAND_SQUARE_FEET_1,801","LAND_SQUARE_FEET_5,466","LAND_SQUARE_FEET_1,380","LAND_SQUARE_FEET_1,930","LAND_SQUARE_FEET_3,338","LAND_SQUARE_FEET_3,692","LAND_SQUARE_FEET_11,800","LAND_SQUARE_FEET_2,755","LAND_SQUARE_FEET_5,202","LAND_SQUARE_FEET_3,230","LAND_SQUARE_FEET_2,121","LAND_SQUARE_FEET_4,361","LAND_SQUARE_FEET_3,910","LAND_SQUARE_FEET_6,867",GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A7,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
7,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,4090,37,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1328.0,1925.0,1,1,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,4120,18,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1728.0,1935.0,1,0,1,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,4120,20,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1728.0,1935.0,1,0,1,0,0,0,0,0,0,0,0,0,0
42,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,6809,54,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1262.0,1925.0,1,1,0,0,0,0,0,0,0,0,0,0,0
44,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,5495,801,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1325.0,1930.0,1,0,0,1,0,0,0,0,0,0,0,0,0
48,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,7491,24,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2655.0,1955.0,1,0,0,0,1,0,0,0,0,0,0,0,0
55,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,2914,49,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1224.0,1950.0,1,0,1,0,0,0,0,0,0,0,0,0,0
61,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,7918,72,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2001.0,1940.0,1,1,0,0,0,0,0,0,0,0,0,0,0
63,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,12174,26,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1280.0,1925.0,1,1,0,0,0,0,0,0,0,0,0,0,0
78,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,4210,19,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2043.0,1925.0,1,1,0,0,0,0,0,0,0,0,0,0,0


In [30]:


from sklearn.feature_selection import f_regression, SelectKBest

selector = SelectKBest(score_func=f_regression, k=15)
x_train_selected = selector.fit_transform(x_train,y_train)
x_test_selected = selector.transform(x_test)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [31]:
x_train_selected.shape, x_test_selected.shape

((4094, 15), (4094, 15))

In [32]:
all_names = x_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
print(selected_names)

Index(['BOROUGH_1', 'NEIGHBORHOOD_UPPER EAST SIDE (59-79)',
       'NEIGHBORHOOD_UPPER EAST SIDE (79-96)', 'BUILDING_CLASS_AT_PRESENT_A4',
       'ADDRESS_412 AVENUE T', 'ADDRESS_101 EAST 63RD STREET',
       'ADDRESS_23 CORNELIA STREET', 'ADDRESS_49 EAST 68TH STREET',
       'ADDRESS_11 EAST 82ND STREET', 'ADDRESS_16 MORTON STREET',
       'LAND_SQUARE_FEET_2,510', 'LAND_SQUARE_FEET_2,056',
       'LAND_SQUARE_FEET_2,554', 'GROSS_SQUARE_FEET',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A4'],
      dtype='object')


In [0]:
# Feature scaling
from sklearn.preprocessing import scale


In [0]:
x_train = pd.DataFrame(scale(x_train_selected), columns=selected_names)

In [35]:
x_train

,BOROUGH_1,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_UPPER EAST SIDE (79-96),BUILDING_CLASS_AT_PRESENT_A4,ADDRESS_412 AVENUE T,ADDRESS_101 EAST 63RD STREET,ADDRESS_23 CORNELIA STREET,ADDRESS_49 EAST 68TH STREET,ADDRESS_11 EAST 82ND STREET,ADDRESS_16 MORTON STREET,"LAND_SQUARE_FEET_2,510","LAND_SQUARE_FEET_2,056","LAND_SQUARE_FEET_2,554",GROSS_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE_A4
0,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,-0.322036,-0.110062
1,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,0.207967,-0.110062
2,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,0.207967,-0.110062
3,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,-0.409487,-0.110062
4,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,-0.326012,-0.110062
5,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,1.436249,-0.110062
6,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,-0.459837,-0.110062
7,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,0.569694,-0.110062
8,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,-0.385637,-0.110062
9,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,0.625344,-0.110062


In [0]:
x_test = pd.DataFrame(scale(x_test_selected), columns=selected_names)

In [37]:
x_test

,BOROUGH_1,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_UPPER EAST SIDE (79-96),BUILDING_CLASS_AT_PRESENT_A4,ADDRESS_412 AVENUE T,ADDRESS_101 EAST 63RD STREET,ADDRESS_23 CORNELIA STREET,ADDRESS_49 EAST 68TH STREET,ADDRESS_11 EAST 82ND STREET,ADDRESS_16 MORTON STREET,"LAND_SQUARE_FEET_2,510","LAND_SQUARE_FEET_2,056","LAND_SQUARE_FEET_2,554",GROSS_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE_A4
0,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,-0.322036,-0.110062
1,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,0.207967,-0.110062
2,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,0.207967,-0.110062
3,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,-0.409487,-0.110062
4,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,-0.326012,-0.110062
5,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,1.436249,-0.110062
6,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,-0.459837,-0.110062
7,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,0.569694,-0.110062
8,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,-0.385637,-0.110062
9,-0.088757,-0.044248,-0.022108,-0.110062,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.015631,-0.02708,-0.015631,-0.015631,0.625344,-0.110062


In [0]:
y_train = scale(y_train)

In [39]:
y_train

array([-0.51546148, -0.51546148, -0.51546148, ..., -0.39839858,
       -0.51546148, -0.51546148])

In [0]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge().fit(x_train, y_train)

In [41]:
ridge_reg

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [43]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_train, ridge_reg.predict(x_test))

0.3123849685555089

In [44]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression().fit(x_train, y_train)
mean_squared_error(y_train, lin_reg.predict(x_test))

0.3125147260476147

In [0]:
# What happens if you scale a one-hot-encoded dataset